In [1]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper functions for visualization.
%matplotlib inline

You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

clinical = pd.read_csv('data/clinical_data_breast_cancer.csv', index_col=0)
proteomes_orig = pd.read_csv('data/77_cancer_proteomes_CPTAC_itraq.csv')
PAM50 = pd.read_csv('data/PAM50_proteins.csv')
proteomes = proteomes_orig.drop(['gene_symbol','gene_name'], axis=1)
clinical.index = clinical.index.to_series().apply(lambda title : title.split('CGA-')[1])
proteomes.rename(columns = proteomes.columns.to_series().apply(lambda title: title.split('.')[0]), inplace=True)

#Transpose and organize proteomes data
proteomes = proteomes.transpose()
proteomes.columns =  proteomes.iloc[0]
proteomes.drop('RefSeq_accession_number', axis=0, inplace=True)

#Convert gender to numbers
def num_gender(gender):
    if gender == 'MALE':
        return 0
    elif gender == 'FEMALE':
        return 1
    else:
        return float('NaN')

clinical['Gender'] = clinical['Gender'].apply(lambda gender: num_gender(gender))

#Convert status to numbers
def num_status(status):
    if status == 'Negative':
        return 0
    elif status == 'Positive':
        return 1
    else:
        return

clinical['ER Status'] = clinical['ER Status'].apply(lambda status: num_status(status))
clinical['PR Status'] = clinical['PR Status'].apply(lambda status: num_status(status))
clinical['HER2 Final Status'] = clinical['HER2 Final Status'].apply(lambda status: num_status(status))

#Convert tumor, node, metastasis to numbers
clinical['Tumor'] = clinical['Tumor'].apply(lambda tumor: tumor.split('T')[1])
clinical['Node'] = clinical['Node'].apply(lambda tumor: tumor.split('N')[1])
clinical['Metastasis'] = clinical['Metastasis'].apply(lambda tumor: tumor.split('M')[1])
#Remove unused columns
clinical.drop('Tumor--T1 Coded', axis=1, inplace=True)
clinical.drop('Metastasis-Coded', axis=1, inplace=True)
clinical.drop('Node-Coded', axis=1, inplace=True)

#Merge clinical data with proteomes data
dataset = clinical.merge(proteomes, left_index=True,right_index=True)

dataset

,Gender,Age at Initial Pathologic Diagnosis,ER Status,PR Status,HER2 Final Status,Tumor,Node,Metastasis,AJCC Stage,Converted Stage,...,NP_001193600,NP_061134,NP_932347,NP_003593,NP_997203,NP_001191293,NP_775791,NP_004065,NP_068752,NP_219494
A2-A0CM,1,40,0.0,0,0.0,2,0,0,Stage IIA,Stage IIA,...,NaN,NaN,1.153614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2-A0D2,1,45,0.0,0,0.0,2,0,0,Stage IIB,Stage IIA,...,0.919136,-1.648856,0.832649,NaN,-8.324969,-4.679219,NaN,-1.10665,NaN,-6.941181
A2-A0EQ,1,64,0.0,0,1.0,2,0,0,Stage IIA,Stage IIA,...,-0.801685,NaN,NaN,3.80231,-6.373934,-1.12316,NaN,NaN,NaN,NaN
A2-A0EV,1,80,1.0,1,0.0,1,0,0,Stage IA,Stage I,...,-4.966177,-1.471027,NaN,-0.474013,-12.278546,-10.337729,-0.653251,NaN,NaN,NaN
A2-A0EX,1,46,1.0,1,0.0,3,0,0,Stage IIB,Stage IIB,...,1.45149,-2.018981,0.877456,NaN,NaN,-6.101005,NaN,-1.726336,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C8-A138,1,54,1.0,0,1.0,2,2,0,Stage III,Stage IIIA,...,-3.250913,1.711825,-0.248402,NaN,4.707022,4.107251,-3.20337,1.971481,NaN,NaN
D8-A142,1,74,0.0,0,0.0,3,0,0,Stage IIB,Stage IIB,...,-5.107629,-0.97598,NaN,2.508629,-12.33711,-9.54653,-4.066584,NaN,NaN,NaN
E2-A154,1,68,1.0,1,0.0,1,0,0,Stage I,Stage I,...,-3.386203,-2.328692,-2.806642,NaN,-4.733495,-9.584499,-4.786183,-3.103949,NaN,NaN
E2-A158,1,43,0.0,0,0.0,1,1,0,Stage IIA,Stage IIA,...,-0.638364,0.051811,2.509998,7.067839,NaN,0.378972,NaN,NaN,0.665797,NaN


In [3]:
proteomesCols= dataset.columns[26:]
proteomsFrame = dataset[proteomesCols]
proteomsFrame

,NP_958782,NP_958785,NP_958786,NP_000436,NP_958781,NP_958780,NP_958783,NP_958784,NP_112598,NP_001611,...,NP_001193600,NP_061134,NP_932347,NP_003593,NP_997203,NP_001191293,NP_775791,NP_004065,NP_068752,NP_219494
A2-A0CM,0.683404,0.694424,0.698098,0.687077,0.687077,0.698098,0.698098,0.698098,-2.65215,-0.984373,...,NaN,NaN,1.153614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2-A0D2,0.107491,0.104164,0.107491,0.097512,0.104164,0.104164,0.104164,0.104164,-0.880454,-1.512473,...,0.919136,-1.648856,0.832649,NaN,-8.324969,-4.679219,NaN,-1.10665,NaN,-6.941181
A2-A0EQ,-0.91267,-0.927979,-0.927979,-0.931806,-0.927979,-0.927979,-0.927979,-0.927979,-3.071151,-2.278943,...,-0.801685,NaN,NaN,3.80231,-6.373934,-1.12316,NaN,NaN,NaN,NaN
A2-A0EV,0.452986,0.47259,0.47259,0.458587,0.47259,0.47259,0.47259,0.47259,-0.742871,1.811277,...,-4.966177,-1.471027,NaN,-0.474013,-12.278546,-10.337729,-0.653251,NaN,NaN,NaN
A2-A0EX,1.185108,1.192612,1.18886,1.185108,1.200116,1.18886,1.18886,1.192612,1.046289,2.138081,...,1.45149,-2.018981,0.877456,NaN,NaN,-6.101005,NaN,-1.726336,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C8-A138,2.765081,2.779709,2.779709,2.797995,2.787023,2.779709,2.783366,2.783366,2.205538,0.749997,...,-3.250913,1.711825,-0.248402,NaN,4.707022,4.107251,-3.20337,1.971481,NaN,NaN
D8-A142,0.538596,0.542211,0.542211,0.534981,0.542211,0.542211,0.542211,0.542211,-0.148205,0.26749,...,-5.107629,-0.97598,NaN,2.508629,-12.33711,-9.54653,-4.066584,NaN,NaN,NaN
E2-A154,0.862659,0.870186,0.870186,0.866423,0.870186,0.870186,0.870186,0.870186,1.920171,2.349197,...,-3.386203,-2.328692,-2.806642,NaN,-4.733495,-9.584499,-4.786183,-3.103949,NaN,NaN
E2-A158,-1.086529,-1.095492,-1.095492,-1.095492,-1.095492,-1.093252,-1.093252,-1.093252,0.096627,-1.149272,...,-0.638364,0.051811,2.509998,7.067839,NaN,0.378972,NaN,NaN,0.665797,NaN


In [6]:
#pass dataframe into pytorch dataset from https://towardsdatascience.com/deep-learning-on-dataframes-with-pytorch-66b21be54ef6
import csv
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
import numpy as np
import re
from pandas.api.types import is_string_dtype, is_numeric_dtype
import warnings
from pdb import set_trace
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.init import *

class ColumnarDataset(Dataset):
    """Dataset class for column dataset.
    Args:
       cats (list of str): List of the name of columns contain
                           categorical variables.
       conts (list of str): List of the name of columns which
                           contain continuous variables.
       y (Tensor, optional): Target variables.
       is_reg (bool): If the task is regression, set ``True``,
                      otherwise (classification) ``False``.
       is_multi (bool): If the task is multi-label classification,
                        set ``True``.
    """
    def __init__(self, df):
        df_cont = df
        conts = [c.values for n,c in df_cont.items()]


        self.conts = np.stack(conts, 1).astype(np.float32)
                           # if conts else np.zeros((n,1))

    def __len__(self): return len(self.conts)
    def __getitem__(self, idx):
        return [self.conts[idx]]


datasetPyTorch = ColumnarDataset(proteomsFrame)
train_loader = torch.utils.data.DataLoader(datasetPyTorch, batch_size=1, shuffle=True)
for data in train_loader:
	print(data)



[tensor([[-0.0782, -0.0681, -0.0714,  ...,     nan,     nan,     nan]])]
[tensor([[0.4530, 0.4726, 0.4726,  ...,    nan,    nan,    nan]])]
[tensor([[-1.0012, -1.0046, -1.0046,  ...,     nan,     nan,     nan]])]
[tensor([[ 1.4076,  1.4076,  1.4103,  ..., -0.9337,     nan,     nan]])]
[tensor([[1.1749, 1.1832, 1.1832,  ...,    nan,    nan,    nan]])]
[tensor([[1.1013, 1.1013, 1.0978,  ...,    nan,    nan,    nan]])]
[tensor([[0.2385, 0.2498, 0.2442,  ...,    nan,    nan,    nan]])]
[tensor([[ 1.3952,  1.4089,  1.4123,  ..., -2.3756,     nan,     nan]])]
[tensor([[ 0.8627,  0.8702,  0.8702,  ..., -3.1039,     nan,     nan]])]
[tensor([[0.3113, 0.2962, 0.2962,  ...,    nan,    nan,    nan]])]
[tensor([[-0.4878, -0.4878, -0.4878,  ...,     nan,     nan,     nan]])]
[tensor([[ 0.3505,  0.3674,  0.3674,  ..., -1.3938, -3.0571,     nan]])]
[tensor([[-1.9478, -1.9527, -1.9552,  ...,     nan,     nan,     nan]])]
[tensor([[-1.1017, -1.1088, -1.1088,  ...,  1.4925, -4.0986,     nan]])]
[tensor(

In [19]:
#getting only columns with no Na values
noNaProteasomes= torch.tensor(proteomsFrame.dropna(axis=1).astype('float32').values)

In [23]:
from torch_cluster import knn_graph
edge_index = knn_graph(noNaProteasomes, k=18, loop=False)
edge_index

tensor([[15, 78,  1,  ..., 64,  8, 27],
        [ 0,  0,  0,  ..., 79, 79, 79]])

In [25]:
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing


class PointNetLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        # Message passing with "max" aggregation.
        super().__init__(aggr='max')

        # Initialization of the MLP:
        # Here, the number of input features correspond to the hidden node
        # dimensionality plus point dimensionality (=3).
        self.mlp = Sequential(Linear(in_channels + 3, out_channels),
                              ReLU(),
                              Linear(out_channels, out_channels))

    def forward(self, h, pos, edge_index):
        # Start propagating messages.
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self, h_j, pos_j, pos_i):
        # h_j defines the features of neighboring nodes as shape [num_edges, in_channels]
        # pos_j defines the position of neighboring nodes as shape [num_edges, 3]
        # pos_i defines the position of central nodes as shape [num_edges, 3]

        input = pos_j - pos_i  # Compute spatial relation.

        if h_j is not None:
            # In the first layer, we may not have any hidden node features,
            # so we only combine them in case they are present.
            input = torch.cat([h_j, input], dim=-1)

        return self.mlp(input)  # Apply our final MLP.

In [ ]:
import torch
import torch.nn.functional as F
from torch_cluster import knn_graph
from torch_geometric.nn import global_max_pool


class PointNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        torch.manual_seed(12345)
        self.conv1 = PointNetLayer(3, 32)
        self.conv2 = PointNetLayer(32, 32)
        self.classifier = Linear(32, dataset.num_classes)

    def forward(self, pos, batch):
        # Compute the kNN graph:
        # Here, we need to pass the batch vector to the function call in order
        # to prevent creating edges between points of different examples.
        # We also add `loop=True` which will add self-loops to the graph in
        # order to preserve central point information.
        edge_index = knn_graph(pos, k=16, batch=batch, loop=True)

        # 3. Start bipartite message passing.
        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(h=h, pos=pos, edge_index=edge_index)
        h = h.relu()

        # 4. Global Pooling.
        h = global_max_pool(h, batch)  # [num_examples, hidden_channels]

        # 5. Classifier.
        return self.classifier(h)


model = PointNet()
print(model)